#### This notebook is necessary in order to correctly obtain the wallet address that is making the withdrawal.

As there is the possibility of making a withdrawal through a relayer, the information about the wallet that is making the withdrawal is not stored in the field "from_address" (in the field "to_address" there are always the addresses of the tornado_cash contracts), but it is stored as one of the parameters of the withdrawal function, which is coded in the field "input".

In this notebook, Web3 library is used to decode the information stored in input, given the ABI of the contracts, and to build the complete dataset.

In [1]:
from web3 import Web3
import pandas as pd

In [2]:
tornado_withdraw_df = pd.read_csv("../data/withdraw_transactions.csv")
tornado_addresses_df = pd.read_csv("../data/tornadocontracts_abi.csv",names=["address", "token", "value", "name","abi"],sep="|")

In [3]:
# Just so that python can understand the contracts abi.
false = False
true = True

In [4]:
tornado_withdraw_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
0,0x8211d6815cdd607a1f8a0d5db08b3cd0121d6df4c6f5...,0,54,0x00058bf70d45aa62a493e5fc596974edde903324,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,421760,37000000000,0x21a0adb6000000000000000000000000000000000000...,2727540,...,NaN,NaN,1,2020-07-16 04:21:43 UTC,10468347,0x97002c5d89010843864301297092eb60b3d902f1850d...,NaN,NaN,NaN,37000000000
1,0xcc8746c16a2bf379e8319b7c190676b77508f09f7a9c...,800,56,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0,423207,13000000000,0x21a0adb6000000000000000000000000000000000000...,3032779,...,NaN,NaN,1,2019-12-16 20:06:53 UTC,9117186,0x8dc6ade4626b61074d088ba600ddcc2686bdc1795681...,NaN,NaN,NaN,13000000000
2,0x9bb7303af6ce69085abc3d9f4f5b7884a90023fd6e59...,799,51,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0,423207,13000000000,0x21a0adb6000000000000000000000000000000000000...,3003518,...,NaN,NaN,1,2019-12-16 20:04:26 UTC,9117176,0xb6c3554845e041c2de52409bbb4b83ede32d98a68a34...,NaN,NaN,NaN,13000000000
3,0xebf94d3b12838665fdd3fccd747d1f74a1686df6c94a...,4,31,0x009e10e12b02db0d56d5f7e1fdc91df98a654f1e,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,20000000000000000,405048,55147597003,0x21a0adb6000000000000000000000000000000000000...,2663888,...,NaN,NaN,1,2021-01-10 14:51:51 UTC,11627774,0xd6efacdd985f8cf384e05838737f2038198d2e2e0b8e...,NaN,NaN,NaN,55147597003
4,0x2708ff9854543a52bdd203aa94b3131bbdd7c268b34d...,3,116,0x00a78b13233caf7750483ada217d8c16ecf4e7bf,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,423183,13000000000,0x21a0adb6000000000000000000000000000000000000...,5244683,...,NaN,NaN,1,2020-01-19 12:23:53 UTC,9311742,0x5565d4dbe84ad74ba8d9b5a3f1b7af741b5c67f0082f...,NaN,NaN,NaN,13000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22004,0x57f9557dc94a1ea540159a60e739cce07acc6ead3630...,3,135,0xff3f5f31fcb7b1d1cc81cb510d6062705da42180,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,421784,30000000000,0x21a0adb6000000000000000000000000000000000000...,11697398,...,NaN,NaN,1,2020-11-10 12:33:21 UTC,11229831,0xd22916dfd059bb7dca3da421fcdb83a7efa785870a00...,NaN,NaN,NaN,30000000000
22005,0x7c03cafbbb6c030b60b24836936e42a6308f80062d57...,16,25,0xff6513ae7efefb2ccc9eda37448082d5b81bd3c4,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,379400,61500000000,0x21a0adb6000000000000000000000000000000000000...,1802082,...,NaN,NaN,1,2020-12-16 12:40:16 UTC,11464189,0x553f06a284a5f40c33452fd569cbb7efd8b08c44dd46...,NaN,NaN,NaN,61500000000
22006,0xddeb6ed2666c7b72136dc6cce20dcd5356295eba08b7...,547,75,0xff80d0aba54b1b89547e029a76fe93e7807d8ee7,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,421772,12000000000,0x21a0adb6000000000000000000000000000000000000...,7240418,...,NaN,NaN,1,2020-05-24 21:28:30 UTC,10130996,0x0f96615108ad530e632572f08c838310a01e0e862e92...,NaN,NaN,NaN,12000000000
22007,0x59cc7d0df8172f091e1c47bc29b9fbe367512f9cfdc0...,1,20,0xff9d2238b57b74caf368cab629acb47258c09974,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,423195,11000000000,0x21a0adb6000000000000000000000000000000000000...,943607,...,NaN,NaN,1,2020-02-28 00:16:24 UTC,9568843,0x7b3a052df4dfc5cb8d38f342ad2b560fabe865f78e4a...,NaN,NaN,NaN,11000000000


In [5]:
def tornado_contracts(tornado_df):
    #creates contract object given it address and abi
    w3 = Web3(Web3.HTTPProvider("https://cloudflare-eth.com"))
    tornado_contracts_dic = {}
    for index, row in tornado_df.iterrows():
        tornado_contracts_dic[row["address"]] = w3.eth.contract(address=w3.toChecksumAddress(row["address"]), abi=eval(row["abi"]))
    return tornado_contracts_dic

In [6]:
tornado_contracts_dic = tornado_contracts(tornado_addresses_df)

In [7]:
tornado_contracts_dic

{'0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3': <web3._utils.datatypes.Contract at 0x7fa27f4810a0>,
 '0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144': <web3._utils.datatypes.Contract at 0x7fa27f481040>,
 '0x07687e702b410fa43f4cb4af7fa097918ffd2730': <web3._utils.datatypes.Contract at 0x7fa27f3f6160>,
 '0x23773e65ed146a459791799d01336db287f25334': <web3._utils.datatypes.Contract at 0x7fa27f433d00>,
 '0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc': <web3._utils.datatypes.Contract at 0x7fa27f3b4ac0>,
 '0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936': <web3._utils.datatypes.Contract at 0x7fa27f3f06a0>,
 '0x910cbd523d972eb0a6f4cae4618ad62622b39dbf': <web3._utils.datatypes.Contract at 0x7fa27f3bffd0>,
 '0xa160cdab225685da1d56aa342ad8841c3b53f291': <web3._utils.datatypes.Contract at 0x7fa27f3d8790>,
 '0xd96f2b1c14db8458374d9aca76e26c3d18364307': <web3._utils.datatypes.Contract at 0x7fa27ef54970>,
 '0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d': <web3._utils.datatypes.Contract at 0x7fa27ef80310>,
 '0x169ad2

In [8]:
#Lets see how contract.decode_function_input works
func_obj, func_params = tornado_contracts_dic[tornado_withdraw_df.iloc[0]["to_address"]].decode_function_input(tornado_withdraw_df.iloc[0]["input"])

In [9]:
func_obj

<Function withdraw(bytes,bytes32,bytes32,address,address,uint256,uint256)>

In [10]:
func_params #in this case _recipient and _relayer are the same because the withdrawer did not request to use a relayer (the wallet could pay for the gas price).

{'_proof': b'\x00U\xde\xf7\xd2\x06d.\xa9B5B\tx\xff\x0e\xa8\x1f\xd1^}\x8d\x7f\x9a\xb2\xb2a\xbf\x98\x0b\xb1\xd3!\xe7\x89\x94\xd9\xd0\xc4\x94\x08\xa2*t\x81h\x1fyE\xd8\xbe&\xef\xc2>\x18:A!i\xdaz\xf4B/\xb1s1\x91)\xd3{}\xb5\xdf\x1d\xbb\x9fo]\xb6\x0f\x1e_\x81\xab\x07\xf4\xf00\xda\xc1\xce\x83\x99\x9e,\xba\x94\x9a\x94\xb2-\x1fJ\xf9t\x9fF0m\xb1n\xa5U\xce\xa7A)Q"/\x7f\xc5qn\xae]\n5\x8b\xd5\xd1Rq6\x13~\xc46\xcbg\x14\x9a,\xa9\x984#Y\xf4\xf2\xbc\x89\x04\xd1\xae[R\x05"\xb4\xb6\xdb\xcd\xfe\xc5!~0\xfc\xd6\x07\x97\xcf`\x1b\xd7{;\x0e\x91W\xe2\xe23\x14\x08\xb2U:4\x0eL\x89p\x07#I\x8cn0\xc1\xcag\xa9\xa5\xd0\xe6J0\x13\x84_&)\x99i\xe7\xc8$\xaa\xcfn"\xac\x16\xd0dS4"\x96D=\x0cz7\xbe\xa8\x87\xff\x98\xcb\x08\x88H\x15%\xa0\xcb\xc7auM\x9d',
 '_root': b'&,[u!\x9f\xb5\x9b:nS]\x04\xd7\xf4\xdd\xf7\xb8\xff\xd3#bc\xba\xbe\xba\xf0\xe3Rq}\xb2',
 '_nullifierHash': b"'\x82\xf2\x8d\xda\x1b\xfc\xcf\x8b\x1cY\xd0YZ@\x02\x14\xa3+\xda\xd8\xdc\xd3\xb1u:@\x13\xa8\xd6US",
 '_recipient': '0x00058BF70d45AA62A493E5FC596974EdDe903324',
 

In [11]:
def recipient_address(row, tornado_contracts_dic):
    func_obj, func_params = tornado_contracts_dic[row["to_address"]].decode_function_input(row["input"])
    return func_params["_recipient"]

In [12]:
tornado_withdraw_df["recipient_address"] = tornado_withdraw_df.apply (lambda row: recipient_address(row, tornado_contracts_dic), axis=1)

In [13]:
tornado_withdraw_df #now we have the dataframe with the column "recipient_address"

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,recipient_address
0,0x8211d6815cdd607a1f8a0d5db08b3cd0121d6df4c6f5...,0,54,0x00058bf70d45aa62a493e5fc596974edde903324,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,421760,37000000000,0x21a0adb6000000000000000000000000000000000000...,2727540,...,NaN,1,2020-07-16 04:21:43 UTC,10468347,0x97002c5d89010843864301297092eb60b3d902f1850d...,NaN,NaN,NaN,37000000000,0x00058BF70d45AA62A493E5FC596974EdDe903324
1,0xcc8746c16a2bf379e8319b7c190676b77508f09f7a9c...,800,56,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0,423207,13000000000,0x21a0adb6000000000000000000000000000000000000...,3032779,...,NaN,1,2019-12-16 20:06:53 UTC,9117186,0x8dc6ade4626b61074d088ba600ddcc2686bdc1795681...,NaN,NaN,NaN,13000000000,0x0039F22efB07A647557C7C5d17854CFD6D489eF3
2,0x9bb7303af6ce69085abc3d9f4f5b7884a90023fd6e59...,799,51,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0,423207,13000000000,0x21a0adb6000000000000000000000000000000000000...,3003518,...,NaN,1,2019-12-16 20:04:26 UTC,9117176,0xb6c3554845e041c2de52409bbb4b83ede32d98a68a34...,NaN,NaN,NaN,13000000000,0x0039F22efB07A647557C7C5d17854CFD6D489eF3
3,0xebf94d3b12838665fdd3fccd747d1f74a1686df6c94a...,4,31,0x009e10e12b02db0d56d5f7e1fdc91df98a654f1e,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,20000000000000000,405048,55147597003,0x21a0adb6000000000000000000000000000000000000...,2663888,...,NaN,1,2021-01-10 14:51:51 UTC,11627774,0xd6efacdd985f8cf384e05838737f2038198d2e2e0b8e...,NaN,NaN,NaN,55147597003,0x5456cB66af7b3F4515C4e11Bbd84ce047A96f9Cf
4,0x2708ff9854543a52bdd203aa94b3131bbdd7c268b34d...,3,116,0x00a78b13233caf7750483ada217d8c16ecf4e7bf,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,423183,13000000000,0x21a0adb6000000000000000000000000000000000000...,5244683,...,NaN,1,2020-01-19 12:23:53 UTC,9311742,0x5565d4dbe84ad74ba8d9b5a3f1b7af741b5c67f0082f...,NaN,NaN,NaN,13000000000,0x00A78b13233CAF7750483aDa217d8C16eCf4e7bf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22004,0x57f9557dc94a1ea540159a60e739cce07acc6ead3630...,3,135,0xff3f5f31fcb7b1d1cc81cb510d6062705da42180,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,421784,30000000000,0x21a0adb6000000000000000000000000000000000000...,11697398,...,NaN,1,2020-11-10 12:33:21 UTC,11229831,0xd22916dfd059bb7dca3da421fcdb83a7efa785870a00...,NaN,NaN,NaN,30000000000,0xFF3F5f31FcB7B1D1CC81Cb510d6062705DA42180
22005,0x7c03cafbbb6c030b60b24836936e42a6308f80062d57...,16,25,0xff6513ae7efefb2ccc9eda37448082d5b81bd3c4,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,379400,61500000000,0x21a0adb6000000000000000000000000000000000000...,1802082,...,NaN,1,2020-12-16 12:40:16 UTC,11464189,0x553f06a284a5f40c33452fd569cbb7efd8b08c44dd46...,NaN,NaN,NaN,61500000000,0x91E0969a9456574982C15Ad96150F7085226a9Cd
22006,0xddeb6ed2666c7b72136dc6cce20dcd5356295eba08b7...,547,75,0xff80d0aba54b1b89547e029a76fe93e7807d8ee7,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,421772,12000000000,0x21a0adb6000000000000000000000000000000000000...,7240418,...,NaN,1,2020-05-24 21:28:30 UTC,10130996,0x0f96615108ad530e632572f08c838310a01e0e862e92...,NaN,NaN,NaN,12000000000,0xFf80D0aba54b1b89547E029A76Fe93e7807d8eE7
22007,0x59cc7d0df8172f091e1c47bc29b9fbe367512f9cfdc0...,1,20,0xff9d2238b57b74caf368cab629acb47258c09974,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,423195,11000000000,0x21a0adb6000000000000000000000000000000000000...,943607,...,NaN,1,2020-02-28 00:16:24 UTC,9568843,0x7b3a052df4dfc5cb8d38f342ad2b560fabe865f78e4a...,NaN,NaN,NaN,11000000000,0xff9D2238B57b74Caf368caB629ACB47258C09974


In [14]:
# tornado_withdraw_df.to_csv("../data/tornado_withdraw_df.csv")